In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from gtts import gTTS
import pygame

def run_realtime_recognition(model_path):
    mp_hands = mp.solutions.hands
    model = load_model(model_path)
    labels = np.load("labels.npy")

    pygame.init()
    pygame.mixer.init()

    sequence = []

    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(max_num_hands=1) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image)

            if results.multi_hand_landmarks:
                hand_landmarks = results.multi_hand_landmarks[0]
                keypoints = []
                for lm in hand_landmarks.landmark:
                    keypoints.extend([lm.x, lm.y, lm.z])
                sequence.append(keypoints)

                if len(sequence) == 30:
                    input_data = np.expand_dims(sequence, axis=0)
                    prediction = model.predict(input_data)
                    class_id = np.argmax(prediction)
                    sentence = labels[class_id]
                    print("Detected:", sentence)

                    tts = gTTS(text=sentence, lang='en')
                    tts.save("output.mp3")
                    pygame.mixer.music.load("output.mp3")
                    pygame.mixer.music.play()

                    sequence = []

            cv2.imshow("ASL Detection", frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
